## Variable Labeling Ratio with blunt tip.

In [1]:
%matplotlib inline

from pathlib import Path
import sys
sys.path.append("../../../../")

import anamic

import numpy as np
import matplotlib.pyplot as plt
import tqdm
from joblib import Parallel, delayed

data_dir = Path('/home/hadim/Drive/Data/Postdoc/Fake Microtubules/anamic/accuracy')
data_dir.mkdir(parents=True, exist_ok=True)

# Consider moving this to `anamic`.
def worker(i_mt, basename, variable_parameter_name,
           variable_parameter, current_dir,
           mt_args, image_args):
    
    image_args[variable_parameter_name] = variable_parameter
    
    # Generate the dimers
    dimers = anamic.simulator.dimers_builder(**mt_args)
    
    # Create the simulator object
    ms = anamic.simulator.MicrotubuleSimulator(dimers)
    
    # Update the parameters
    ms.parameters.update(image_args)
        
    # Build a single microtubule image
    ms.build_all(apply_random_z_rotation=True, show_progress=False)
    
    # Round image
    ms.image = np.round(ms.image)
    ms.image = ms.image.astype('uint16')
    
    # Save image, parameters and dimer positions.
    ms.save_positions(current_dir / (basename + ".csv"))
    ms.save_parameters(current_dir / (basename + ".json"))
    ms.save_image(current_dir / (basename + ".tif"))
    
    return ms

In [2]:
experiment_name = 'labeling_ratio_with_blunt_tip_small'

variable_parameter_name = 'labeling_ratio'
variable_parameters = [0.1, 0.4, 0.7, 1.0]

experiment_dir = data_dir / experiment_name
experiment_dir.mkdir(parents=True, exist_ok=True)

n_mt_per_condition = 3

# Set parameters for the MT simulation
mt_args = {}
mt_args['n_pf'] = 13
mt_args['mt_length_nm'] = 2000  # nm
mt_args['taper_length_nm'] = 0  # nm

# Set parameters for the image generation.
image_args = {}
image_args['labeling_ratio'] = 0.1  # from 0 to 1

image_args['pixel_size'] = 110  # nm/pixel
image_args['x_offset'] = 4500  # nm
image_args['y_offset'] = 4500  # nm

image_args['psf_size'] = 130  # nm
image_args['signal_mean'] = 700
image_args['signal_std'] = 100
image_args['bg_mean'] = 500
image_args['bg_std'] = 24
image_args['noise_factor'] = 1.2

# Generate the workers arguments
worker_args = []
for variable_parameter in variable_parameters:
    current_dir = experiment_dir / f'{variable_parameter_name}-{variable_parameter:.1f}'
    current_dir.mkdir(parents=True, exist_ok=True)
    for i in range(n_mt_per_condition):
        worker_arg = {}
        worker_arg['i_mt'] = i
        worker_arg['basename'] = f'microtubule_{i:04d}'
        worker_arg['variable_parameter_name'] = variable_parameter_name
        worker_arg['variable_parameter'] = variable_parameter
        worker_arg['current_dir'] = current_dir
        worker_arg['mt_args'] = mt_args.copy()
        worker_arg['image_args'] = image_args.copy()
        worker_args.append(worker_arg)

In [5]:
anamic.utils.parallelized(worker, worker_args, args_type='dict', progress_bar=True, n_jobs=-1)